<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

  Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Macro


In [ ]:
import language.experimental.macros

// Normal imports
import reflect.macros.blackbox.Context

object Step1Complete {
  def hello(): Unit = macro hello_impl

  def hello_impl(c: Context)(): c.Expr[Unit] = {
    import c.universe._
    println("Compiling!")
    reify { println("Hello World!") }
  }
     
}      

## quasiquotes 
https://docs.scala-lang.org/overviews/quasiquotes/setup.html  
Compared with LISP's macro

In [ ]:
val universe: scala.reflect.runtime.universe.type = scala.reflect.runtime.universe
import universe._
val tree = q"i am { a quasiquote }"
val C = q"class C"
//println(showCode(C))
//println(showCode(tree))
//println(showRaw(tree))

In [ ]:
val a = q"true"
val b = true

In [ ]:
val q"i am $what" = q"i am { a quasiquote }"

In [ ]:
val ab = List(q"a", q"b")

In [ ]:
import scala.reflect.runtime.currentMirror
import scala.tools.reflect.ToolBox
val toolbox = currentMirror.mkToolBox()
val fab = q"f(..$ab)"
//val C = q"class C"
val tree = q"i am { a quasiquote }"
println(tree match { case q"i am { a quasiquote }" => "it worked!" })

In [ ]:
println(q"foo + bar" equalsStructure q"foo.+(bar)")

In [ ]:
val x = q"""
         val x: List[Int] = List(1, 2) match {
           case List(a, b) => List(a + b)
         }
       """

In [ ]:
val q"f(..$args)" = q"f(a, b)"

In [ ]:
val two = 1 + 1
val four = q"$two + $two"
val ints = List(1, 2, 3)
val f123 = q"f(..$ints)"

In [ ]:
val q"${left: Int} + ${right: Int}" = q"2 + 2"

In [ ]:
val q"..$stats" = q"(2,3)"

In [ ]:
val code = q"""println("compiled and run at runtime!")"""
val compiledCode = toolbox.compile(code)
val result = compiledCode()

In [ ]:
val x = q"package mypackage { class MyClass }"
showCode(x)

In [ ]:
 
  def generateCode() =
    q"package mypackage { class MyClass }"
  def saveToFile(path: String, code: Tree) = {
    val writer = new java.io.PrintWriter(path)
    try writer.write(showCode(code))
    finally writer.close()
  }
  saveToFile("myfile.scala", generateCode())
 

In [ ]:
val q"$expr.$tname" = q"a.eat"
val q"${x: scala.Symbol}" = q"'a"


In [ ]:
val q"$mods def $tname[..$tparams](...$paramss): $tpt = $expr" = q"""
private def interrupt(namePre:String, triggers:Int*)(name:String): Unit =  
println("hello scala macro")
"""

In [ ]:
val q"..$stats0" = q"a; b; c"
val q"..$stats1" = q"(a, b, c)"
val q"..$stats2" = q"""{a
b
c}"""
val q"..$stats3" = q"""{
val a: Int = 1
val b: Int = 2
val c = 3}"""

In [ ]:
 val extractor = pq"Foo(1, 2, 3)"
 val extractor2 = q"Foo(1, 2, 3)"
val pq"$id(..$pats)" = extractor
val pq"$id2(..$pats2)" = extractor2

In [ ]:
val isT =  pq"_: T"

In [ ]:
val f3 = q"private implicit def f"

In [ ]:
import language.experimental.macros

// Normal imports
import reflect.macros.blackbox.Context

In [ ]:
abstract class Animal {
  def name: String
}
case class Cat(name: String) extends Animal
case class Dog(name: String) extends Animal

In [ ]:
object CovarianceTest extends App {
  def printAnimalNames(animals: List[Animal]): Unit = {
    animals.foreach { animal =>
      println(animal.name)
    }
  }

  val cats: List[Cat] = List(Cat("Whiskers"), Cat("Tom"))
  val dogs: List[Dog] = List(Dog("Fido"), Dog("Rex"))

  printAnimalNames(cats)
  // Whiskers
  // Tom

  printAnimalNames(dogs)
  // Fido
  // Rex
}

In [ ]:
//CovarianceTest.printAnimalNames(CovarianceTest.cats)
 val cats: List[Cat] = List(Cat("Whiskers"), Cat("Tom"))
  val dogs: List[Dog] = List(Dog("Fido"), Dog("Rex"))
CovarianceTest.printAnimalNames(cats)
    CovarianceTest.printAnimalNames(dogs)

In [ ]:
println(HTMLFactory)

In [ ]:
val htmlbody = "html5 ${body} end"
val body = "-body-"
StringContext("html5 ","end").s(body)

In [ ]:
val body = "div xxx div "
val b = s"html5 ${body} end" 
val c = new StringContext(htmlbody).s(htmlbody)
val d = StringContext("html5 ${body} end")
 
//val bf = "html5 %s end".format(body)

In [ ]:
class T1 extends Component{
    val f = UInt"11000011"
}
showRtl(new T1)

In [ ]:
import scala.reflect.macros.whitebox.Context
import scala.language.experimental.macros
import scala.annotation.StaticAnnotation
object helloMacro {
  def impl(c: Context)(annottees: c.Expr[Any]*): c.Expr[Any] = {
    import c.universe._
    val result = {
      annottees.map(_.tree).toList match {
        case ModuleDef(mods, name, Template(parents, self, body)) :: Nil =>
          val helloMethod = DefDef(NoMods, TermName("hello"), List(), List(List()), TypeTree(), Literal(Constant("hello")))
          ModuleDef(mods, name, Template(parents, self, body :+ helloMethod))
      }
    }
    c.Expr[Any](result)
  }
}

In [ ]:
import scala.reflect.macros.blackbox
object dataMacro {
  def impl(c: blackbox.Context)(annottees: c.Expr[Any]*): c.Expr[Any] = {
    import c.universe._

    val result = {
      annottees.map(_.tree).toList match {
        case q"""
              class $name {
                ..$vars
              }
              """ :: Nil =>

          // Generate the Getter and Setter from VarDefs
          val beanMethods = vars.collect {
            case q"$mods var $name: $tpt = $expr" =>
              val getName = TermName("get" + name.encodedName.toString.capitalize)
              val setName = TermName("set" + name.encodedName.toString.capitalize)
              println(getName)
              val ident = Ident(name)
              List (
                q"def $getName: $tpt = $ident",
                q"def $setName(paramX: $tpt): Unit = { $ident = paramX }"
              )
          }.flatten

          // Insert the generated Getter and Setter
          q"""
             class $name {
               ..$vars
               ..$beanMethods
             }
           """
        case _ =>
          throw new Exception("Macro Error")
      }
    }
    c.Expr[Any](result)
  }
}


In [ ]:
class data extends StaticAnnotation {
  def macroTransform(annottees: Any*): Any = macro dataMacro.impl
}